In [54]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community langchainhub sentence-transformers

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [7]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [15]:
print(docs[0])

page_content='

      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-t

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
for i in range(5):
    print(f"{i} : ", splits[i]) 

0 :  page_content='LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory' metadata={

In [19]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition refers to the process of breaking down complex tasks into smaller, more manageable steps. This technique is often facilitated by the Chain of Thought (CoT) prompting method, which encourages models to "think step by step." By doing so, it enhances performance on difficult tasks and provides insight into the model\'s reasoning process.'

# 여기서부터 실습 시작

In [51]:
loader = WebBaseLoader(
    web_path="https://applied-llms.org/",
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("content")
        )
    ),
)

docs = loader.load()

docs[0]

Document(metadata={'source': 'https://applied-llms.org/'}, page_content='\n\n\nWhat We‚Äôve Learned From A Year of Building with LLMs\n\n\n\n    A practical guide to building successful LLM products, covering the tactical, operational, and strategic.\n  \n\n\n\nAuthors\n\nEugene Yan \nBryan Bischof \nCharles Frye \nHamel Husain \nJason Liu \nShreya Shankar \n\n\n\nPublished\n\nJune 8, 2024\n\n\n\n\n\nAlso published on O‚ÄôReilly Media in three parts: Tactical, Operational, Strategic (podcast). Also translated to Japanese (by Kazuya Kanno)\n\nIt‚Äôs an exciting time to build with large language models (LLMs). Over the past year, LLMs have become ‚Äúgood enough‚Äù for real-world applications. And they‚Äôre getting better and cheaper every year. Coupled with a parade of demos on social media, there will be an estimated $200B investment in AI by 2025. Furthermore, provider APIs have made LLMs more accessible, allowing everyone, not just ML engineers and scientists, to build intelligence in

In [52]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(docs)

print(splits[0])

page_content='What We‚Äôve Learned From A Year of Building with LLMs



    A practical guide to building successful LLM products, covering the tactical, operational, and strategic.
  



Authors

Eugene Yan 
Bryan Bischof 
Charles Frye 
Hamel Husain 
Jason Liu 
Shreya Shankar 



Published

June 8, 2024





Also published on O‚ÄôReilly Media in three parts: Tactical, Operational, Strategic (podcast). Also translated to Japanese (by Kazuya Kanno)' metadata={'source': 'https://applied-llms.org/'}


In [55]:
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(docs, embedding_function)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/Users/ray/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

InvalidDimensionException: Embedding dimension 384 does not match collection dimensionality 1536

In [130]:
user_question = "RAG 에 대한 저자의 생각은 무엇인가? RAG 와 fine tuning 에 대해 저자는 어떻게 비교하고 있나? 저자가 가장 많은 부분을 할당해 설명하는 개념은 무엇인가?"
#user_question = "오늘 날씨는 어때?"
#user_question = "LLM의 RAG와 연체동물의 RAG에 대한 차이는 뭐라고 생각하고 있지?"

In [131]:
first_answer = rag_chain.invoke(user_question)
print(first_answer)

The provided context does not contain specific information about the author's thoughts on RAG or how they compare it to fine-tuning. It primarily discusses a categorization of human memory in relation to sensory, short-term, and long-term memory. Therefore, I don't know the answer to the question.


In [124]:
from langchain.prompts.prompt import PromptTemplate
# step 'relevance check'

prompt = PromptTemplate(
    template="""
Please check if the answer to the question is unknown.
If you answered 'I don't know', say 'no'. If you answered 'I know', say 'yes'.

<question>
{question}

<system_answer>
{system_answer}

<answer>
{answer}
""",
    input_variables=["question", "system_answer"],
    partial_variables={"answer":"answer"},
)

chain = prompt | llm

step1_answer = chain.invoke({"question" : user_question, "system_answer" : first_answer}).content

print(step1_answer)

yes


In [125]:
if step1_answer.lower() == "yes":
    # 이후 스텝
    pass
else:
    # 종료
    print("No!")

In [134]:
# step 'hallucination check'

prompt_template = """
The following prompts can be used to identify hallucinations in LLM's responses:
CopyYou are a professional reviewing LLM's responses to identify hallucinations or inaccurate information. Please carefully analyze the following response and rate it according to the following guidelines:

1. Fact Checking: Verify all facts, dates, statistics, and quotes provided in the response.

2. Logical Consistency: Ensure that the response is consistent throughout.

3. Contextual Appropriateness: Evaluate whether the response is appropriate to the context of the question.

4. Expressing Uncertainty: Ensure that LLM has appropriately expressed uncertainty about information that is uncertain.

5. Cite Sources: Ensure that sources are provided for important information.

6. Detect Contradictions: Identify statements that contradict one another in the response.

7. Overgeneralization: Check for inappropriate overgeneralization.

Score each item (1-5, with 5 being the highest) and answer 'yes' if the total score is over 29, otherwise 'no'.

Your answer should be 'yes' or 'no' only.

Response from LLM to be reviewed:

{llm_response}
"""

prompt = PromptTemplate.from_template(prompt_template)

chain = prompt | llm

step2_answer = chain.invoke({"llm_response" : first_answer}).content

print(step2_answer)

Score for each item:

1. Fact Checking: 5 (The response does not present any specific facts, dates, or statistics that need verification.)
2. Logical Consistency: 5 (The response is logically consistent, stating clearly that it lacks the necessary information to answer the question.)
3. Contextual Appropriateness: 5 (The response appropriately addresses the context of the question by acknowledging the lack of information.)
4. Expressing Uncertainty: 5 (The LLM effectively expresses uncertainty by stating "I don't know the answer to the question.")
5. Cite Sources: 1 (No sources are provided, but given the lack of specific information, this may not be applicable.)
6. Detect Contradictions: 5 (There are no contradictions in the response.)
7. Overgeneralization: 5 (There are no inappropriate overgeneralizations present.)

Total Score: 31

Answer: yes
